In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

proximity = "./data/0_Golden_Standard/Country_Borders_source_target_V06.tsv"
flights = "./data/0_Golden_Standard/country_flight_country_weighted.tsv"
covid = "./data/0_Golden_Standard/covid_data_01.csv"

In [2]:
pwd

'/home/jan/Code/MasterThesis'

In [3]:
prt = pd.read_csv(proximity,sep='\t')
flt = pd.read_csv(flights,sep='\t')
cov = pd.read_csv(covid,sep=',')

In [4]:
prt = prt.rename(columns={"iso-alpha-3":"iso_code", "Country": "src", "Neighbour":"trg"})
prt

,src,trg,iso_code,region
0,Afghanistan,China,AFG,Asia
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia
2,Afghanistan,Pakistan,AFG,Asia
3,Afghanistan,Tajikistan,AFG,Asia
4,Afghanistan,Turkmenistan,AFG,Asia
...,...,...,...,...
791,Zambia,Zimbabwe,ZMB,Africa
792,Zimbabwe,Botswana,ZWE,Africa
793,Zimbabwe,Mozambique,ZWE,Africa
794,Zimbabwe,South Africa,ZWE,Africa


In [5]:
prt.loc[prt['iso_code'] == 'MDA']

,src,trg,iso_code,region
465,"Moldova, Republic of",Romania,MDA,Europe
466,"Moldova, Republic of",Ukraine,MDA,Europe


In [6]:
flt

,src,trg,weighted
0,Afghanistan,Azerbaijan,1
1,Afghanistan,India,7
2,Afghanistan,Iran,5
3,Afghanistan,Kuwait,1
4,Afghanistan,Pakistan,4
...,...,...,...
4553,Zimbabwe,Malawi,1
4554,Zimbabwe,Namibia,2
4555,Zimbabwe,South Africa,9
4556,Zimbabwe,Tanzania,1


In [7]:
cov

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30859,NaN,NaN,International,2020-02-28,705.0,0.0,4.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30860,NaN,NaN,International,2020-02-29,705.0,0.0,6.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30861,NaN,NaN,International,2020-03-01,705.0,0.0,6.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30862,NaN,NaN,International,2020-03-02,705.0,0.0,6.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#drop irrelevant columns
cov = cov.drop(["new_cases", 
                "total_deaths",
                "new_deaths",
                "new_cases_per_million",
                "total_deaths_per_million",
                "new_deaths_per_million",
                "total_tests",
                "new_tests",
                "total_tests_per_thousand",
                "new_tests_per_thousand",
                "new_tests_smoothed",
                "new_tests_smoothed_per_thousand",
                "tests_units",
                "extreme_poverty",
                "cvd_death_rate",
                "population"], axis=1)

#set 999 days date for countries in which CoVid did not hit
#cov[['date']] = cov[['date']].fillna(pd.to_datetime("2022-10-13", format='%Y-%m-%d'))

#Removing non-country related values

cov = cov[cov.location != "International"]
cov = cov[cov.iso_code.map(len) <= 3]
cov = cov[cov.total_cases > 100]
cov = cov[cov.total_cases <= 500]
cov=cov.drop_duplicates(subset=['location'],keep="first")

#drop potentially useful columns
cov = cov.drop(["location",
                "population_density", 
                "median_age",
                "aged_65_older",
                "aged_70_older",
                "gdp_per_capita",
                "female_smokers",
                "male_smokers",
                "handwashing_facilities",
                "hospital_beds_per_thousand",
                "life_expectancy",
                "diabetes_prevalence"], axis=1)

#building target variable
cov[['date']] = cov['date'].fillna("2020-12-30")
cov['total_cases'] = cov['total_cases'].fillna("0")
cov = cov.rename(columns={"stringency_index": "days_since_100_cases_in_China_2019"})

cov['date'] = pd.to_datetime(cov['date'], format='%Y-%m-%d')


cov['date']


79      2020-03-29
206     2020-03-25
397     2020-03-21
525     2020-03-23
723     2020-06-11
           ...    
30061   2020-03-23
30241   2020-06-28
30297   2020-05-16
30403   2020-05-01
30549   2020-05-28
Name: date, Length: 180, dtype: datetime64[ns]

In [9]:
date1 = "2020-01-19"
date_obj= pd.to_datetime("2020-01-18", format='%Y-%m-%d')

cov["days_since_100_cases_in_China_2019"] = cov.apply(lambda row: row.date - date_obj, axis=1)

#dropping date, total_cases and total_cases_per_m and continent
cov_u = cov.drop(["date",
                "total_cases", 
                "total_cases_per_million",
                "continent"], axis=1)

cov_u


,iso_code,days_since_100_cases_in_China_2019
79,AFG,71 days
206,ALB,67 days
397,DZA,63 days
525,AND,65 days
723,AGO,145 days
...,...,...
30061,VNM,65 days
30241,ESH,162 days
30297,YEM,119 days
30403,ZMB,104 days


In [10]:
cov_u.sort_values(by ='days_since_100_cases_in_China_2019')[0:20]

,iso_code,days_since_100_cases_in_China_2019
5923,CHN,1 days
25639,KOR,34 days
14580,JPN,35 days
14255,ITA,37 days
13343,IRN,40 days
10740,DEU,43 days
24783,SGP,43 days
25952,ESP,44 days
9975,FRA,44 days
29228,USA,45 days


In [11]:
#cov_u[['days_since_100_cases_in_China_2019']] = cov_u[['days_since_100_cases_in_China_2019']].fillna(pd.Timedelta('999 days'))

#u = cov_u.select_dtypes(include=['datetime'])
#cov_u[u.columns] = u.fillna(pd.to_datetime('today'))
#df

In [12]:
set_iso = set(prt['src'])
set_fl = set(flt['src'])

In [13]:
def to_be_changed(t1,t2):
    to_add = [i for i in t2 if i not in t1]
    to_review = [i for i in t1 if i not in t2]
    return sorted(to_add), sorted(to_review)

In [14]:
# Common Replacements to set ISO 3166 standard
rep = {'British Virgin Islands': 'Virgin Islands (British)',
  'Burma': 'Myanmar',
  'Cape Verde' : 'Cabo Verde',
  'Congo (Brazzaville)': 'Congo',
  'Congo (Kinshasa)': 'Congo, Democratic Republic of the',
  "Cote d'Ivoire": "Côte d'Ivoire",
  'Czech Republic': "Czechia",
  'Falkland Islands':"Falkland Islands (Malvinas)",
  'Iran':"Iran (Islamic Republic of)",
  'Laos':"Lao People's Democratic Republic",
  'Macau': 'Macao',
  'Macedonia': 'North Macedonia',
  'Micronesia': "Micronesia (Federated States of)",
  'Moldova': "Moldova, Republic of",
  'North Korea':"Korea (Democratic People's Republic of)",
  'Reunion': "Réunion",
  'Russia':"Russian Federation",
  'South Korea':"Korea, Republic of",
  'Tanzania':"Tanzania, United Republic of",
  'United Kingdom':"United Kingdom of Great Britain and Northern Ireland",
  'United States':"United States of America",
  'Venezuela':"Venezuela (Bolivarian Republic of)",
  'Vietnam':"Viet Nam",
  'Virgin Islands':"Virgin Islands (British)"}

# Given a dataframes and a replacement list, it searches for the source and target country names in the database 
# and updates its naming with they values from the dict

def common_rep(dfo, dfr):
    for key, value in dfr.items():
        dfo.loc[dfo['src'] == key, 'src'] = value
        dfo.loc[dfo['trg'] == key, 'trg'] = value
    return dfo

flt = common_rep(flt, rep)
to_be_changed(set(prt['src']).union(set(prt['trg'])), set(flt['src']).union(set(flt['trg'])))

([],
 ['Andorra',
  'Antarctica',
  'Antartica',
  'Bonaire, Sint Eustatius and Saba',
  'Bouvet Island',
  'British Indian Ocean Territory',
  'Curaçao',
  'Eswatini',
  'Finalnd',
  'French Southern Territories',
  'Heard Island and McDonald Islands',
  'Holy See',
  'Kosovo',
  'Liechtenstein',
  'Monaco',
  'Montserrat',
  'Palestine, State of',
  'Pitcairn',
  'Saint Barthélemy',
  'Saint Helena, Ascension and Tristan da Cunha',
  'Saint Martin (French part)',
  'San Marino',
  'Sint Maarten (Dutch part)',
  'South Georgia and the South Sandwich Islands',
  'Svalbard and Jan Mayen',
  'Syrian Arab Republic',
  'Timor-Leste',
  'Tokelau',
  'United States Minor Outlying Islands',
  'Virgin Islands (U.S.)',
  'Åland Islands'])

In [15]:
#verifying flt integrity
flt

,src,trg,weighted
0,Afghanistan,Azerbaijan,1
1,Afghanistan,India,7
2,Afghanistan,Iran (Islamic Republic of),5
3,Afghanistan,Kuwait,1
4,Afghanistan,Pakistan,4
...,...,...,...
4553,Zimbabwe,Malawi,1
4554,Zimbabwe,Namibia,2
4555,Zimbabwe,South Africa,9
4556,Zimbabwe,"Tanzania, United Republic of",1


In [16]:
import pandas as pd
from sklearn import preprocessing

# creating log_weighted flights
flt["log-weight"] = np.log(flt["weighted"])

#creating normalized log-weighted flights column

x = flt[["log-weight"]].values.astype(float) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
flt['norm-log-weight'] = pd.DataFrame(x_scaled)
flt

,src,trg,weighted,log-weight,norm-log-weight
0,Afghanistan,Azerbaijan,1,0.000000,0.000000
1,Afghanistan,India,7,1.945910,0.311347
2,Afghanistan,Iran (Islamic Republic of),5,1.609438,0.257511
3,Afghanistan,Kuwait,1,0.000000,0.000000
4,Afghanistan,Pakistan,4,1.386294,0.221808
...,...,...,...,...,...
4553,Zimbabwe,Malawi,1,0.000000,0.000000
4554,Zimbabwe,Namibia,2,0.693147,0.110904
4555,Zimbabwe,South Africa,9,2.197225,0.351557
4556,Zimbabwe,"Tanzania, United Republic of",1,0.000000,0.000000


In [17]:
#Countries with no flights:
no_fly = ['Andorra',
  'Antarctica',
  'Bonaire, Sint Eustatius and Saba',
  'Bouvet Island',
  'British Indian Ocean Territory',
  'Curaçao',
  'Eswatini',
  'French Southern Territories',
  'Heard Island and McDonald Islands',
  'Holy See',
  'Kosovo',
  'Liechtenstein',
  'Monaco',
  'Montserrat',
  'Netherlands Antilles',
  'Palestine, State of',
  'Pitcairn',
  'Saint Barthélemy',
  'Saint Helena, Ascension and Tristan da Cunha',
  'Saint Martin (French part)',
  'San Marino',
  'Sint Maarten (Dutch part)',
  'South Georgia and the South Sandwich Islands',
  'Svalbard and Jan Mayen',
  'Syrian Arab Republic',
  'Timor-Leste',
  'Tokelau',
  'United States Minor Outlying Islands',
  'Virgin Islands (U.S.)',
  'Åland Islands']

def no_air(no_flights, df):
    for e in no_flights:
        indexNames1 = df[df['src'] == e ].index
        indexNames2 = df[df['trg'] == e ].index
        df.drop(indexNames1 , inplace=True)
        df.drop(indexNames2 , inplace=True)
    return df

prt = no_air(no_fly, prt)
to_be_changed(set(prt['src']).union(set(prt['trg'])), set(flt['src']).union(set(flt['trg'])))

(['Netherlands Antilles'], [])

In [18]:
# verifying prt integrity
prt

,src,trg,iso_code,region
0,Afghanistan,China,AFG,Asia
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia
2,Afghanistan,Pakistan,AFG,Asia
3,Afghanistan,Tajikistan,AFG,Asia
4,Afghanistan,Turkmenistan,AFG,Asia
...,...,...,...,...
791,Zambia,Zimbabwe,ZMB,Africa
792,Zimbabwe,Botswana,ZWE,Africa
793,Zimbabwe,Mozambique,ZWE,Africa
794,Zimbabwe,South Africa,ZWE,Africa


In [19]:
#ISO Naming

#we drop target to avoid conflict on merge, only interested on src and iso code's to join with flt_u
prt_named = prt.drop(["trg"], axis=1)

def iso_fill(df1,df2,where):
    df3=pd.merge(df1,df2, on=where, how="outer")
    return df3

In [20]:
#updating flights DB
flt_u = iso_fill(prt_named,flt,["src"]).dropna()
flt_u

,src,iso_code,region,trg,weighted,log-weight,norm-log-weight
0,Afghanistan,AFG,Asia,Azerbaijan,1,0.000000,0.000000
1,Afghanistan,AFG,Asia,India,7,1.945910,0.311347
2,Afghanistan,AFG,Asia,Iran (Islamic Republic of),5,1.609438,0.257511
3,Afghanistan,AFG,Asia,Kuwait,1,0.000000,0.000000
4,Afghanistan,AFG,Asia,Pakistan,4,1.386294,0.221808
...,...,...,...,...,...,...,...
18075,Zimbabwe,ZWE,Africa,Malawi,1,0.000000,0.000000
18076,Zimbabwe,ZWE,Africa,Namibia,2,0.693147,0.110904
18077,Zimbabwe,ZWE,Africa,South Africa,9,2.197225,0.351557
18078,Zimbabwe,ZWE,Africa,"Tanzania, United Republic of",1,0.000000,0.000000


In [21]:
flt_u = iso_fill(flt_u,cov_u,"iso_code")
flt_u = flt_u.drop_duplicates(subset=['src', 'trg'],keep="first")

#fixing corner cases

flt_u["days_since_100_cases_in_China_2019"] = flt_u.days_since_100_cases_in_China_2019.fillna(
    pd.to_datetime("2022-10-13", format='%Y-%m-%d') - pd.to_datetime("2020-01-18", format='%Y-%m-%d'))
flt_u = flt_u.dropna()

In [22]:
# verification
flt_u.isna().values.any()

False

In [23]:
# verification
flt_u.loc[flt_u['src'] == 'Western Sahara']
#flt_u.sort_values(['days_since_100_cases_in_China_2019', 'src'], ascending=[True, False])[-150:]

,src,iso_code,region,trg,weighted,log-weight,norm-log-weight,days_since_100_cases_in_China_2019
17922,Western Sahara,ESH,Africa,Morocco,4.0,1.386294,0.221808,162 days
17923,Western Sahara,ESH,Africa,Spain,3.0,1.098612,0.175779,162 days


In [24]:
# Do the same with countries_proximity
prt_u = iso_fill(prt,cov_u,"iso_code")
prt_u = prt_u.drop_duplicates(subset=['src', 'trg'],keep="first")

#fixing corner cases

prt_u["days_since_100_cases_in_China_2019"] = prt_u.days_since_100_cases_in_China_2019.fillna(
    pd.to_datetime("2022-10-13", format='%Y-%m-%d') - pd.to_datetime("2020-01-18", format='%Y-%m-%d'))
prt_u = prt_u.dropna()

In [25]:
# verification
prt_u.isna().values.any()

False

In [26]:
# verification
#prt_u.isna().values.any()
prt_u.loc[prt_u['iso_code'] == 'MDA']
#prt_u.sort_values(['src', 'days_since_100_cases_in_China_2019'], ascending=[False, True])[-50:]

,src,trg,iso_code,region,days_since_100_cases_in_China_2019
431,"Moldova, Republic of",Romania,MDA,Europe,66 days
432,"Moldova, Republic of",Ukraine,MDA,Europe,66 days


In [27]:
# id'ing a potential missmatch on countries
to_be_changed(set(prt_u['src']).union(set(prt_u['trg'])), set(flt_u['src']).union(set(flt_u['trg'])))

(['Netherlands Antilles'], [])

In [28]:
found_c = prt_u[prt_u['src'].str.contains('Netherlands Antilles')]
found_c += prt_u[prt_u['trg'].str.contains('Netherlands Antilles')]

print(found_c.count())

src                                   0
trg                                   0
iso_code                              0
region                                0
days_since_100_cases_in_China_2019    0
dtype: int64


In [29]:
found_c = flt_u[flt_u['src'].str.contains('Netherlands Antilles')]
found_c += flt_u[flt_u['trg'].str.contains('Netherlands Antilles')]

print(found_c.count())

src                                   0
iso_code                              0
region                                0
trg                                   0
weighted                              0
log-weight                            0
norm-log-weight                       0
days_since_100_cases_in_China_2019    0
dtype: int64


In [30]:
# FIXING EAST TIMOR ISO CODE ERROR
prt_u.loc[prt_u['iso_code'] == 'TL']
prt_u['iso_code'] = prt_u['iso_code'].replace(['TL'],'TLS')
prt_u

,src,trg,iso_code,region,days_since_100_cases_in_China_2019
0,Afghanistan,China,AFG,Asia,71 days
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,71 days
2,Afghanistan,Pakistan,AFG,Asia,71 days
3,Afghanistan,Tajikistan,AFG,Asia,71 days
4,Afghanistan,Turkmenistan,AFG,Asia,71 days
...,...,...,...,...,...
717,Zambia,Zimbabwe,ZMB,Africa,104 days
718,Zimbabwe,Botswana,ZWE,Africa,131 days
719,Zimbabwe,Mozambique,ZWE,Africa,131 days
720,Zimbabwe,South Africa,ZWE,Africa,131 days


In [31]:
from datetime import datetime, timedelta

# Converting from timedelta into integer to avoid saving in csv errors
prt_u["days_since_100_cases_in_China_2019"] = prt_u["days_since_100_cases_in_China_2019"]/(timedelta (days=1))
flt_u["days_since_100_cases_in_China_2019"] = flt_u["days_since_100_cases_in_China_2019"]/(timedelta (days=1))
prt_u

,src,trg,iso_code,region,days_since_100_cases_in_China_2019
0,Afghanistan,China,AFG,Asia,71.0
1,Afghanistan,Iran (Islamic Republic of),AFG,Asia,71.0
2,Afghanistan,Pakistan,AFG,Asia,71.0
3,Afghanistan,Tajikistan,AFG,Asia,71.0
4,Afghanistan,Turkmenistan,AFG,Asia,71.0
...,...,...,...,...,...
717,Zambia,Zimbabwe,ZMB,Africa,104.0
718,Zimbabwe,Botswana,ZWE,Africa,131.0
719,Zimbabwe,Mozambique,ZWE,Africa,131.0
720,Zimbabwe,South Africa,ZWE,Africa,131.0


In [32]:
prt_u["weeks_since_100_cases"] = prt_u["days_since_100_cases_in_China_2019"].astype(int)//7
flt_u[flt_u['src']=="Spain"]

,src,iso_code,region,trg,weighted,log-weight,norm-log-weight,days_since_100_cases_in_China_2019
14746,Spain,ESP,Europe,Algeria,14.0,2.639057,0.422251,44.0
14747,Spain,ESP,Europe,Angola,1.0,0.000000,0.000000,44.0
14748,Spain,ESP,Europe,Argentina,4.0,1.386294,0.221808,44.0
14749,Spain,ESP,Europe,Austria,40.0,3.688879,0.590223,44.0
14750,Spain,ESP,Europe,Belarus,1.0,0.000000,0.000000,44.0
...,...,...,...,...,...,...,...,...
14814,Spain,ESP,Europe,United Kingdom of Great Britain and Northern I...,512.0,6.238325,0.998136,44.0
14815,Spain,ESP,Europe,United States of America,57.0,4.043051,0.646891,44.0
14816,Spain,ESP,Europe,Uruguay,1.0,0.000000,0.000000,44.0
14817,Spain,ESP,Europe,Venezuela (Bolivarian Republic of),4.0,1.386294,0.221808,44.0


In [33]:
#verifying that only China is patient 0
prt_u[prt_u['weeks_since_100_cases']==0]

,src,trg,iso_code,region,days_since_100_cases_in_China_2019,weeks_since_100_cases
141,China,Afghanistan,CHN,Asia,1.0,0
142,China,Bhutan,CHN,Asia,1.0,0
143,China,Hong Kong,CHN,Asia,1.0,0
144,China,India,CHN,Asia,1.0,0
145,China,Kazakhstan,CHN,Asia,1.0,0
146,China,Korea (Democratic People's Republic of),CHN,Asia,1.0,0
147,China,Kyrgyzstan,CHN,Asia,1.0,0
148,China,Lao People's Democratic Republic,CHN,Asia,1.0,0
149,China,Macao,CHN,Asia,1.0,0
150,China,Mongolia,CHN,Asia,1.0,0


In [34]:
prt_u.to_csv('./data/0_Golden_Standard/c_proximity_clean_04.csv')
flt_u.to_csv('./data/0_Golden_Standard/c_flights_clean_04.csv')